**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from class_utils import error_histogram

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("sigmoid_regression_data.csv"), directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Rozhodovacie stromy pre regresiu

Rozhodovacie stromy je možné aplikovať na klasifikáciu aj na regresiu. Ukážeme preto jeden priamočiary príklad aplikácie na regresnú úlohu. Dátová množina bude 2-rozmerná, čo nám umožní ju ľahko zobraziť a vizuálne zhodnotiť regresnú krivku. Zdrojový kód potrebný na načítanie a predspracovanie dát je k dipozícii v nasledujúcej bunke a kvôli stručnosti je skrytý.



In [ ]:
#@title -- Loading and Preprocessing the Data: X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
df = pd.read_csv("data/sigmoid_regression_data.csv")

kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['y']])

df_train, df_test = train_test_split(
    df, stratify=y_stratify, test_size=0.3, random_state=4)

plt.scatter(df_train['x'], df_train['y'], marker='x', label="training data")
plt.scatter(df_test['x'], df_test['y'], c='r', label="testing data")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()

categorical_inputs = []
numeric_inputs = ['x']
output = 'y'

input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values

### Vytvorenie a tréning regresora

Na vytvorenie regresora potrebujeme jednoducho vytvoriť inštanciu triedy `DecisionTreeRegressor`. Budeme používať predvolené parametre, takže už v tejto fáze nemusíme špecifikovať nič ďalšie. Tréning sa bude realizovať pomocou štandardného rozhrania `fit`.



In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, Y_train)

### Testovanie

Testovanie vykonáme určením strednej kvadratickej chyby (mean squared error; MSE) a strednej absolútnej chyby (mean absolute error; MAE) na testovacej množine. Zobrazíme tiež histogram chýb, aby sme získali lepšiu predstavu o tom, ako sú chyby rozdelené a aká je ich škála voči pôvodným dátam.



In [ ]:
#@title -- Testing -- { display-mode: "form" }
y_test = model.predict(X_test)

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_test, y_test)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_test, y_test)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_test, y_test, Y_fit_scaling=Y_train)

Výsledky by mali byť vcelku dobré: chyby by mali byť v porovnaní so škálou dát zanedbateľné.

### Vizualizácia regresnej krivky

Keď vizualizujeme regresnú krivku, všimneme si, že nie je hladká, ale po častiach konštantná. Je to tak preto, že každému listu stromu zodpovedá určitá oblasť vo vstupnom priestore a každý list predikuje jednu konštantnú hodnotu. V dôsledku toho je vstupný priestor rozdelený na diskrétne oblasti s rôznymi konštantnými výstupmi.



In [ ]:
#@title -- Regression Curve vs. Data -- { display-mode: "form" }
x_min = min(np.min(X_train), np.min(X_test))
x_max = max(np.max(X_train), np.max(X_test))

xx = np.linspace(x_min, x_max, 250).reshape((-1, 1)).astype(np.float32)
yy = model.predict(xx)

plt.scatter(X_train, Y_train, marker='x', label="training data")
plt.scatter(X_test, Y_test, c='r', label="testing data")

plt.plot(xx, yy, label="regression curve", c='k')

plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()